## 1. This patient may have sepsis
<p>Sepsis is a deadly syndrome where a patient has a severe infection that causes organ failure. The sooner septic patients are treated, the more likely they are to survive, but sepsis can be challenging to recognize. It may be possible to use hospital data to develop machine learning models that could flag patients who are likely to be septic. However, before we develop predictive algorithms, we need a reliable method to determine patients who are septic. One component of sepsis is a severe infection.</p>
<p>In this project, we will use two weeks of hospital electronic health record (EHR) data to find out which patients had a severe infection according to four criteria. We will look into the data to see if a doctor ordered a blood test to look for bacteria (a blood culture) and gave the patient a series of intervenous antibiotics. </p>
<p>Let's get started!</p>

In [2]:
# Load packages
import pandas as pd

# Read in the data
antibioticDT = pd.read_csv("datasets/antibioticDT.csv")

# Look at the first 30 rows
antibioticDT.head(30)


,patient_id,day_given,antibiotic_type,route
0,1,2,ciprofloxacin,IV
1,1,4,ciprofloxacin,IV
2,1,6,ciprofloxacin,IV
3,1,7,doxycycline,IV
4,1,9,doxycycline,IV
5,1,15,penicillin,IV
6,1,16,doxycycline,IV
7,1,18,ciprofloxacin,IV
8,8,1,doxycycline,PO
9,8,2,penicillin,IV


## 2. Which antibiotics are "new"?
<p>These data represent all drugs administered in a hospital over two weeks. Each row represents one time a patient was given an antibiotic. The variables include the patient identification number, the day the drug was administered, the name of the antibiotic, and how it was administered. For example, patient "8" received doxycycline by mouth on the first day of their stay.</p>
<p>We will identify patients with a serious infection using the following criteria. </p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</li>
</ol>
<p>Let's start with the second item by finding which rows represent "new antibiotics". We will determine if each antibiotic was given to the patient in the prior two days. We'll visualize this task by looking at the data sorted by id, antibiotic type, and day.</p>

In [9]:
# Sort the data and examine the first 40 rows
# Use sort_values() instead of setorder() for Python
antibioticDT_sorted = antibioticDT.sort_values(['patient_id', 'antibiotic_type', 'day_given']).reset_index(drop=True)
antibioticDT_sorted.head(40)

# Use shift to calculate the last day a particular drug was administered
antibioticDT_sorted['last_administration_day'] = antibioticDT_sorted.groupby(['patient_id', 'antibiotic_type'])['day_given'].shift(1)

# Calculate the number of days since the drug was last administered
antibioticDT_sorted['days_since_last_admin'] = antibioticDT_sorted['day_given'] - antibioticDT_sorted['last_administration_day']

# Create antibiotic_new with an initial value of one, then reset it to zero as needed
antibioticDT_sorted['antibiotic_new'] = 1
# Reset to zero if the drug was administered in the previous 2 days (1 or 2 days)
antibioticDT_sorted.loc[(antibioticDT_sorted['days_since_last_admin'] <= 2) & (antibioticDT_sorted['days_since_last_admin'] >= 1), 'antibiotic_new'] = 0

## 3. Looking at the blood culture data
<p>Now let's look at blood culture data from the same two-week period in this hospital. These data are in <code>blood_cultureDT.csv</code>. Let's start by reading it into the workspace and having a look at a few rows. </p>
<p>Each row represents one blood culture and gives the patient's id and the day the blood culture test occurred. For example, patient "8" had a blood culture on the second day of their hospitalization and again on the thirteenth day. Notice that some patients from the antibiotic dataset are not in this dataset and vice versa. Some patients are in neither because they received neither antibiotics nor a blood culture.</p>

In [10]:
# Read in blood_cultureDT.csv
blood_cultureDT = pd.read_csv("datasets/blood_cultureDT.csv")

# Print the first 30 rows
blood_cultureDT.head(30)

,patient_id,blood_culture_day
0,1,3
1,1,13
2,8,2
3,8,13
4,23,3
5,39,10
6,45,4
7,45,9
8,45,11
9,51,3


## 4. Combine the antibiotic data and the blood culture data
<p>To find which antibiotics were given close to a blood culture test, we need to combine the drug administration data with the blood culture data. We'll keep only patients that are still candidates for infection&mdash;only those in both data sets.</p>
<p>A challenge with the data is that some patients had blood cultures on several different days. For each of those days, we will see if there is a sequence of antibiotic days close to them. To accomplish this, in the merge we will match each blood culture to each antibiotic day.</p>
<p>After sorting the data following the merge, you will see that each patient's antibiotic sequence repeats for each blood culture day. This repetition allows us to look at each blood culture day and check if it is associated with a qualifying sequence of antibiotics.</p>

In [11]:
# Merge antibioticDT with blood_cultureDT
# Use merge() to combine the datasets on patient_id
combinedDT = antibioticDT.merge(blood_cultureDT, on='patient_id', how='inner')

# Sort by patient_id, blood_culture_day, day_given, and antibiotic_type
combinedDT = combinedDT.sort_values(['patient_id', 'blood_culture_day', 'day_given', 'antibiotic_type']).reset_index(drop=True)

# Print and examine the first 30 rows
combinedDT.head(30)

,patient_id,day_given,antibiotic_type,route,blood_culture_day
0,1,2,ciprofloxacin,IV,3
1,1,4,ciprofloxacin,IV,3
2,1,6,ciprofloxacin,IV,3
3,1,7,doxycycline,IV,3
4,1,9,doxycycline,IV,3
5,1,15,penicillin,IV,3
6,1,16,doxycycline,IV,3
7,1,18,ciprofloxacin,IV,3
8,1,2,ciprofloxacin,IV,13
9,1,4,ciprofloxacin,IV,13


## 5. Determine whether each row is in-window
<p>Now that we have the antibiotic and blood culture data combined, we can test each drug administration against each blood culture to see if it's "in the window."</p>

In [12]:
# Make a new variable called drug_in_bcx_window
# Calculate the absolute difference between blood culture day and antibiotic day
combinedDT['days_from_bc'] = abs(combinedDT['blood_culture_day'] - combinedDT['day_given'])

# Create drug_in_bcx_window: 1 if within 2 days of blood culture, 0 otherwise
combinedDT['drug_in_bcx_window'] = (combinedDT['days_from_bc'] <= 2).astype(int)

## 6. Check the I.V. requirement
<p>Now let's look at the fourth item in the criteria. </p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li><em>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</em></li>
</ol>

In [14]:
# Create a variable indicating if there was at least one I.V. drug given in the window
# First, create a flag for IV drugs that are in the window
combinedDT['iv_in_window'] = ((combinedDT['drug_in_bcx_window'] == 1) & (combinedDT['route'] == 'I.V.')).astype(int)

# Group by patient and blood culture day to check if any IV drugs were given in the window
combinedDT['iv_in_window_flag'] = combinedDT.groupby(['patient_id', 'blood_culture_day'])['iv_in_window'].transform('max')

# Exclude rows in which the blood_culture_day does not have any I.V. drugs in window 
combinedDT = combinedDT[combinedDT['iv_in_window_flag'] == 1]

## 7. Find the first day of possible sequences
<p>We're getting close! Let's review the criteria again.</p>
<p><strong>Criteria for Suspected Infection</strong><a href="https://www.ncbi.nlm.nih.gov/pubmed/28903154">*</a></p>
<ol>
<li>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</li>
<li>The sequence must start with a new antibiotic, defined as an antibiotic type that was not given in the previous two days.</li>
<li>The sequence must start within two days of a blood culture.</li>
<li>There must be at least one intervenous (I.V.) antibiotic within the +/-2 day window.</li>
</ol>
<p>Let's assess the first criterion by finding the first day of possible 4-day qualifying sequences.    </p>

In [18]:
# Create a new variable called day_of_first_new_abx_in_window
# First, we need to merge the antibiotic_new information from our sorted data
combinedDT = combinedDT.merge(
    antibioticDT_sorted[['patient_id', 'antibiotic_type', 'day_given', 'antibiotic_new']], 
    on=['patient_id', 'antibiotic_type', 'day_given'], 
    how='left'
)

# Create a flag for qualifying rows (in window AND new antibiotic)
combinedDT['qualifying_row'] = (
    (combinedDT['drug_in_bcx_window'] == 1) & 
    (combinedDT['antibiotic_new'] == 1)
).astype(int)

# Find the first qualifying day for each patient/blood culture combination
first_days = (
    combinedDT.loc[combinedDT['qualifying_row'] == 1]
    .groupby(['patient_id', 'blood_culture_day'])['day_given']
    .min()
    .rename('day_of_first_new_abx_in_window')
    .reset_index()
)

# Merge back the first qualifying day
combinedDT = combinedDT.merge(first_days, on=['patient_id', 'blood_culture_day'], how='left')

# Replace missing values with large number (999) for groups without qualifying rows
combinedDT['day_of_first_new_abx_in_window'] = combinedDT['day_of_first_new_abx_in_window'].fillna(999)

# Remove rows where the day is before this first qualifying day
combinedDT = combinedDT[combinedDT['day_given'] >= combinedDT['day_of_first_new_abx_in_window']]


## 8. Simplify the data
<p>The first criterion is: <em>The patient receives antibiotics for a sequence of four days, with gaps of one day allowed.</em></p>
<p>We've pinned down the first day of possible sequences in the previous task. Now we have to check for four-day sequences. We don't need the drug type (name); we need the days the antibiotics were administered.</p>

In [19]:
# Create a new data.table containing only patient_id, blood_culture_day, and day_given
simplified_data = combinedDT[['patient_id', 'blood_culture_day', 'day_given']].copy()

# Remove duplicate rows
simplified_data = simplified_data.drop_duplicates()

## 9. Extract first four rows for each blood culture
<p>To check for four-day sequences, let's pull out the first four days (rows) for each patient/blood culture combination. Some patients will have less than four antibiotic days. We'll remove them first.</p>

In [20]:
# Count the antibiotic days within each patient/blood culture day combination
# Group by patient and blood culture day, then count the number of unique antibiotic days
day_counts = simplified_data.groupby(['patient_id', 'blood_culture_day']).size().reset_index(name='antibiotic_day_count')

# Remove blood culture days with less than four rows 
simplified_data = simplified_data.merge(day_counts, on=['patient_id', 'blood_culture_day'], how='left')
simplified_data = simplified_data[simplified_data['antibiotic_day_count'] >= 4]

# Select the first four days for each blood culture
first_four_days = simplified_data.groupby(['patient_id', 'blood_culture_day']).head(4) 

## 10. Consecutive sequence
<p>Now we need to check whether each four-day sequence qualifies by having no gaps of more than one day.</p>
<!--"Patient receives antibiotics for a sequence of 4 days, with gaps of 1 day allowed."-->

In [25]:
# Make the indicator for consecutive sequence
# Sort the first four days by day_given to ensure chronological order
first_four_days = first_four_days.sort_values(
    ['patient_id', 'blood_culture_day', 'day_given']
).reset_index(drop=True)

# Calculate the differences between consecutive days for each patient/blood culture combination
first_four_days['day_diff'] = first_four_days.groupby(
    ['patient_id', 'blood_culture_day']
)['day_given'].diff()

# Find which four-day sequences qualify
# Use max() of the day_diff: if the maximum gap <= 1, then the sequence has no skips of more than one day
four_in_seq = first_four_days.groupby(['patient_id', 'blood_culture_day'])['day_diff'].transform(
    lambda x: 1 if x.dropna().max() <= 1 else 0
)

# Add the result directly as a new column
first_four_days['four_in_seq'] = four_in_seq



## 11. Select the patients who meet criteria
<p>A patient would meet the criteria if any of their blood cultures were accompanied by a qualifying sequence of antibiotics. Now that we've determined which each blood culture qualify let's select the patients who meet the criteria.</p>

In [26]:
# Select the rows which have four_in_seq equal to 1
suspected_infection = first_four_days[first_four_days['four_in_seq'] == 1]

# Retain only the patient_id column
suspected_infection = suspected_infection[['patient_id']]

# Remove duplicates
suspected_infection = suspected_infection.drop_duplicates()

# Make an infection indicator
suspected_infection['infection_indicator'] = 1

## 12. Find the prevalence of sepsis
<p>In this project, we used two EHR datasets to flag patients who were suspected of having a serious infection. We also got a <code>data.table</code> workout!</p>
<p>So far, we've been looking at records of all antibiotics administered and blood cultures that occurred over two weeks at a particular hospital. However, not all patients who were hospitalized over this period are represented in <code>combinedDT</code> because not all of them took antibiotics or had blood culture tests. We have to read in and merge the rest of the patient information to see what percentage of patients at the hospital might have had a serious infection.</p>

In [27]:
# Read in "all_patients.csv"
all_patientsDT = pd.read_csv("datasets/all_patients.csv")

# Merge this with the infection flag data
all_patientsDT = all_patientsDT.merge(suspected_infection, on='patient_id', how='left')

# Set any missing values of the infection flag to 0
all_patientsDT['infection_indicator'] = all_patientsDT['infection_indicator'].fillna(0)

# Calculate the percentage of patients who met the criteria for presumed infection
ans = (all_patientsDT['infection_indicator'].sum() / len(all_patientsDT)) * 100
print(f"Percentage of patients with suspected infection: {ans:.2f}%")

Percentage of patients with suspected infection: 0.00%
